## Cleaning Data
This week, we'll be cleaning data.



Cleaning data is labor intensive and requires making subjective choices.  

We'll focus on, and assess you on, manipulating data correctly, making reasonable

choices, and documenting the choices you make carefully.



We'll focus on the programming tools that get used in cleaning data in class

this week:

reshaping data
handling missing or incorrect values

In [1]:
import pandas as pd
import seaborn as sns

sns.set_theme(font_scale=2)
arabica_data_url = 'https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/arabica_data_cleaned.csv'

Read in the three csv files described below and store them in a list of dataFrames

In [2]:
url_base = 'https://raw.githubusercontent.com/rhodyprog4ds/rhodyds/main/data/'

datasets = ['study_a.csv','study_b.csv','study_c.csv']

In [3]:
df_list = [pd.read_csv(url_base + file,na_values= '') for file in datasets]

In [4]:
df_list[0]

,name,treatmenta,treatmentb
0,John Smith,-,2
1,Jane Doe,16,11
2,Mary Johnson,3,1


In [5]:
df_list[1]

,intervention,John Smith,Jane Doe,Mary Johnson
0,treatmenta,—,16,3
1,treatmentb,2,11,1


In [6]:
df_list[2]

,person,treatment,result
0,John Smith,a,—
1,Jane Doe,a,16
2,Mary Johnson,a,3
3,John Smith,b,2
4,Jane Doe,b,11
5,Mary Johnson,b,1


In [7]:
treat_df = df_list[0]

In [8]:
treat_df.head()

,name,treatmenta,treatmentb
0,John Smith,-,2
1,Jane Doe,16,11
2,Mary Johnson,3,1


In [9]:
treat_df.columns

Index(['name', ' treatmenta', ' treatmentb'], dtype='object')

In [10]:
treat_df.melt(value_vars = [' treatmenta',' treatmentb'],
             id_vars = ['name'],
             value_name = 'result', var_name = 'treatment' )

,name,treatment,result
0,John Smith,treatmenta,-
1,Jane Doe,treatmenta,16
2,Mary Johnson,treatmenta,3
3,John Smith,treatmentb,2
4,Jane Doe,treatmentb,11
5,Mary Johnson,treatmentb,1


In [11]:
tidy_treat_df = treat_df.melt(value_vars = [' treatmenta',' treatmentb'],
             id_vars = ['name'],
             value_name = 'result', var_name = 'treatment' )

In [12]:
tidy_treat_df.

SyntaxError: invalid syntax (162367199.py, line 1)

In [13]:
tidy_treat_df.groupby('name').mean()

/opt/hostedtoolcache/Python/3.7.12/x64/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


""
name
Jane Doe
John Smith
Mary Johnson


In [14]:
coffee_df = pd.read_csv(arabica_data_url, index_col = 0)
coffee_df.head()

,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,Region,...,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
3,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,NaN,...,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0
4,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,oromia,...,Green,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0
5,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,Green,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0


In [15]:
# compute ___ per ___
bag_total_df = coffee_df.groupby('Country.of.Origin')['Number.of.Bags'].sum()

In [16]:
high_prod_countries =  list(bag_total_df[bag_total_df>15000].index)

In [17]:
high_prod = lambda c: c in high_prod_countries

In [18]:
high_prod('Mexico')

True

In [19]:
high_prod('Ethiopa')

False

In [20]:
coffee_df['high_production'] = coffee_df['Country.of.Origin'].apply(high_prod)

In [21]:
coffee_df.head()

,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,Region,...,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters,high_production
1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0,False
2,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0,False
3,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,NaN,...,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0,True
4,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,oromia,...,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0,False
5,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,guji-hambela,...,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0,False


In [22]:
high_prod_coffee_df = coffee_df[coffee_df['high_production']]

In [23]:
low_prod_coffee_df = coffee_df[coffee_df['high_production']==False]